# 🎙️ Comic Video Pipeline — Stage 3 & 4: TTS + SRT

This notebook generates narration audio using **Qwen3-TTS** (voice cloning) and SRT subtitles using **Whisper**.

**Prerequisites:**
- Run Stage 1 (script) and Stage 2 (images) on your Mac first
- Script and images should be in Google Drive: `comic-pipeline/<project_name>/`
- Prepare a **3-10 second WAV voice sample** for voice cloning

**Flow:**
1. Mount Google Drive
2. Select project & upload voice sample
3. Generate per-scene audio with Qwen3-TTS
4. (Optional) Generate word-level SRT with Whisper
5. Files auto-sync back to your Mac via Drive

## 1️⃣ Setup: Install Dependencies & Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

DRIVE_BASE = '/content/drive/MyDrive/comic-pipeline'

import os
os.makedirs(DRIVE_BASE, exist_ok=True)
print('✅ Google Drive mounted')
print(f'📁 Base path: {DRIVE_BASE}')

# List available projects
projects = [d for d in os.listdir(DRIVE_BASE) 
            if os.path.isdir(os.path.join(DRIVE_BASE, d))
            and os.path.exists(os.path.join(DRIVE_BASE, d, 'script.json'))]
print(f'\n📋 Available projects: {projects}')

In [ ]:
# ===== SET YOUR PROJECT NAME HERE =====
PROJECT_NAME = 'death_of_gwen_stacy'  # <-- CHANGE THIS
# ======================================

PROJECT_DIR = os.path.join(DRIVE_BASE, PROJECT_NAME)
AUDIO_DIR = os.path.join(PROJECT_DIR, 'audio')
os.makedirs(AUDIO_DIR, exist_ok=True)

# Load script
import json
with open(os.path.join(PROJECT_DIR, 'script.json')) as f:
    script = json.load(f)

print(f'📖 Loaded: {script.get("title", PROJECT_NAME)}')
print(f'🎬 Scenes: {len(script["scenes"])}')
for s in script['scenes']:
    print(f'   Scene {s["scene_id"]}: {s["narration"][:60]}...')

## 2️⃣ Install Qwen3-TTS

In [ ]:
%%capture
# Install Qwen3-TTS dependencies
!pip install transformers accelerate soundfile torch torchaudio
!pip install cosyvoice-ttsfrd
print('✅ Dependencies installed')

In [ ]:
import torch
import soundfile as sf
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

print(f'🖥️  GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}')
print(f'💾 VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB' if torch.cuda.is_available() else '')

# Load Qwen3-TTS model
print('\n⏳ Loading Qwen3-TTS model (this may take 2-5 minutes)...')

MODEL_ID = 'Qwen/Qwen3-TTS-12Hz-1.7B-Base'

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True
)
print('✅ Model loaded successfully!')

## 3️⃣ Upload Voice Sample for Cloning

Upload a **3-10 second WAV file** of the voice you want to clone.
For a dramatic comic narrator, use a sample of a deep, authoritative voice.

In [ ]:
from google.colab import files
import torchaudio

# Option 1: Upload from your computer
print('📤 Upload a voice sample WAV file (3-10 seconds):')
uploaded = files.upload()

VOICE_SAMPLE_PATH = None
for filename in uploaded:
    VOICE_SAMPLE_PATH = f'/content/{filename}'
    with open(VOICE_SAMPLE_PATH, 'wb') as f:
        f.write(uploaded[filename])
    break

# Option 2: Use a file already in Google Drive
# VOICE_SAMPLE_PATH = os.path.join(PROJECT_DIR, 'voice_sample.wav')

if VOICE_SAMPLE_PATH and os.path.exists(VOICE_SAMPLE_PATH):
    waveform, sr = torchaudio.load(VOICE_SAMPLE_PATH)
    duration = waveform.shape[1] / sr
    print(f'✅ Voice sample loaded: {duration:.1f}s, {sr}Hz')
else:
    print('⚠️  No voice sample. Will use default voice.')
    VOICE_SAMPLE_PATH = None

## 4️⃣ Generate Narration Audio (Per-Scene)

In [ ]:
def generate_speech(text, voice_sample_path=None, output_path='output.wav'):
    """
    Generate speech from text using Qwen3-TTS.
    If voice_sample_path is provided, clone that voice.
    """
    # Build the prompt for Qwen3-TTS
    if voice_sample_path:
        # Voice cloning mode
        waveform, sr = torchaudio.load(voice_sample_path)
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(sr, 16000)
            waveform = resampler(waveform)
        
        # Prepare prompt with voice reference
        messages = [
            {'role': 'user', 'content': [
                {'type': 'audio', 'audio': voice_sample_path},
                {'type': 'text', 'text': text}
            ]}
        ]
    else:
        messages = [
            {'role': 'user', 'content': text}
        ]
    
    # Generate with the model
    input_text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=4096,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )
    
    # Decode audio tokens to waveform
    audio_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    audio_array = tokenizer.decode_audio(audio_tokens)
    
    # Save
    sf.write(output_path, audio_array, 24000)
    return output_path

print('✅ TTS function ready')

In [ ]:
# ===== GENERATE AUDIO FOR ALL SCENES =====
print('🎙️  Generating narration for all scenes...\n')

generated_files = []

for scene in script['scenes']:
    sid = scene['scene_id']
    narration = scene['narration']
    output_path = os.path.join(AUDIO_DIR, f'scene_{sid:02d}.wav')
    
    print(f'  Scene {sid}: "{narration[:50]}..."')
    
    try:
        generate_speech(
            text=narration,
            voice_sample_path=VOICE_SAMPLE_PATH,
            output_path=output_path
        )
        
        # Verify
        waveform, sr = torchaudio.load(output_path)
        duration = waveform.shape[1] / sr
        print(f'    ✅ Generated: {duration:.1f}s')
        generated_files.append(output_path)
        
    except Exception as e:
        print(f'    ❌ Error: {e}')
        print(f'    Trying without voice cloning...')
        try:
            generate_speech(
                text=narration,
                voice_sample_path=None,
                output_path=output_path
            )
            generated_files.append(output_path)
            print(f'    ✅ Generated (default voice)')
        except Exception as e2:
            print(f'    ❌ Failed completely: {e2}')

print(f'\n✅ Generated {len(generated_files)}/{len(script["scenes"])} audio files')
print(f'📁 Saved to: {AUDIO_DIR}')

## 4b️⃣ (Alternative) If Qwen3-TTS doesn't work, use Chatterbox

Uncomment and run the cells below as a fallback.

In [ ]:
# ===== FALLBACK: CHATTERBOX TTS =====
# Uncomment all lines below if Qwen3-TTS fails

# !pip install chatterbox-tts

# import chatterbox
# from chatterbox.tts import ChatterboxTTS
# 
# cb_model = ChatterboxTTS.from_pretrained(device='cuda')
# print('✅ Chatterbox loaded')
# 
# for scene in script['scenes']:
#     sid = scene['scene_id']
#     output_path = os.path.join(AUDIO_DIR, f'scene_{sid:02d}.wav')
#     
#     wav = cb_model.generate(
#         scene['narration'],
#         audio_prompt_path=VOICE_SAMPLE_PATH,
#         exaggeration=0.6,     # dramatic style
#         cfg=0.5,
#         temperature=0.8,
#     )
#     sf.write(output_path, wav.squeeze().cpu().numpy(), 24000)
#     print(f'  ✅ Scene {sid} done')
# 
# print('✅ All scenes generated with Chatterbox')

## 5️⃣ Concatenate All Audio into One Narration Track

In [ ]:
# Concatenate per-scene audio into one narration file
import wave
import glob

scene_files = sorted(glob.glob(os.path.join(AUDIO_DIR, 'scene_*.wav')))
print(f'📎 Found {len(scene_files)} scene audio files')

if scene_files:
    # Read all and concatenate
    all_audio = []
    sample_rate = None
    
    for f in scene_files:
        data, sr = sf.read(f)
        if sample_rate is None:
            sample_rate = sr
        elif sr != sample_rate:
            # Resample if needed
            import librosa
            data = librosa.resample(data, orig_sr=sr, target_sr=sample_rate)
        all_audio.append(data)
        print(f'  {os.path.basename(f)}: {len(data)/sample_rate:.1f}s')
    
    combined = np.concatenate(all_audio)
    narration_path = os.path.join(AUDIO_DIR, 'narration.wav')
    sf.write(narration_path, combined, sample_rate)
    
    total_duration = len(combined) / sample_rate
    print(f'\n✅ Combined narration: {total_duration:.1f}s')
    print(f'💾 Saved to: {narration_path}')
else:
    print('❌ No audio files found!')

## 6️⃣ (Optional) Word-Level SRT with Whisper

This gives you precise word-by-word subtitles instead of scene-level.
Skip this if scene-level SRT (generated by Stage 5 on your Mac) is sufficient.

In [ ]:
# Install Whisper
!pip install openai-whisper

import whisper

print('⏳ Loading Whisper model...')
whisper_model = whisper.load_model('base')  # Options: tiny, base, small, medium, large
print('✅ Whisper loaded')

In [ ]:
# Transcribe narration to get word-level timestamps
narration_path = os.path.join(AUDIO_DIR, 'narration.wav')

if os.path.exists(narration_path):
    print('🔍 Transcribing with Whisper...')
    result = whisper_model.transcribe(
        narration_path,
        word_timestamps=True,
        language='en'
    )
    
    # Save Whisper JSON
    whisper_json_path = os.path.join(AUDIO_DIR, 'whisper_transcript.json')
    with open(whisper_json_path, 'w') as f:
        json.dump(result, f, indent=2)
    print(f'💾 Whisper JSON saved: {whisper_json_path}')
    
    # Generate SRT from Whisper output
    def format_time(seconds):
        h = int(seconds // 3600)
        m = int((seconds % 3600) // 60)
        s = int(seconds % 60)
        ms = int((seconds % 1) * 1000)
        return f'{h:02d}:{m:02d}:{s:02d},{ms:03d}'
    
    srt_entries = []
    for i, seg in enumerate(result['segments'], 1):
        start = format_time(seg['start'])
        end = format_time(seg['end'])
        text = seg['text'].strip()
        srt_entries.append(f'{i}\n{start} --> {end}\n{text}\n')
    
    srt_content = '\n'.join(srt_entries)
    srt_path = os.path.join(AUDIO_DIR, 'whisper_subtitles.srt')
    with open(srt_path, 'w') as f:
        f.write(srt_content)
    
    print(f'📝 Whisper SRT saved: {srt_path}')
    print(f'   Segments: {len(result["segments"])}')
    print(f'\n   Preview:')
    for seg in result['segments'][:3]:
        print(f'   [{seg["start"]:.1f}s - {seg["end"]:.1f}s] {seg["text"].strip()}')
else:
    print('❌ Narration file not found. Run Step 5 first.')

## ✅ Done!

Your files are now in Google Drive and will sync to your Mac:

```
comic-pipeline/<project>/
├── audio/
│   ├── scene_01.wav          ← per-scene audio
│   ├── scene_02.wav
│   ├── ...
│   ├── narration.wav         ← combined narration
│   ├── whisper_transcript.json  ← (optional) Whisper output
│   └── whisper_subtitles.srt    ← (optional) word-level SRT
```

**Next step on your Mac:**
```bash
python -m stages.stage5_video_assembler --project "<project_name>"
```

In [ ]:
# Final summary
print('=' * 60)
print('📊 SUMMARY')
print('=' * 60)
print(f'Project: {PROJECT_NAME}')
print(f'Title: {script.get("title", "N/A")}')
print(f'\nGenerated files in: {AUDIO_DIR}')

for f in sorted(os.listdir(AUDIO_DIR)):
    fpath = os.path.join(AUDIO_DIR, f)
    size_kb = os.path.getsize(fpath) / 1024
    print(f'  📄 {f} ({size_kb:.0f} KB)')

print(f'\n👉 Go back to your Mac and run:')
print(f'   python -m stages.stage5_video_assembler --project "{PROJECT_NAME}"')